<a href="https://colab.research.google.com/github/BYFarag/Education_Inequality/blob/main/Bayan_Farag_DATA_3320_Education_Inequality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

The objective of this project is to address the issue of inequality in educational opportunities among U.S. high schools. Our main focus is on the average performance of students on standardized tests such as the ACT or SAT, which are integral to the college application process. Through our research, we seek to determine whether socioeconomic factors influence school performance on these exams.

## Import libraries

In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno

# Train-test splits
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV

# Model preprocessing
from sklearn.preprocessing import StandardScaler

# Modeling
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Model preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

# Model metrics and analysis
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# Models
from sklearn import linear_model

## The data

This project utilizes two data sets. The primary data set is the EdGap data set from [EdGap.org](https://www.edgap.org/#5/37.875/-96.987). This data set from 2016 includes information about average ACT or SAT scores for schools and several socioeconomic characteristics of the school district. The secondary data set is basic information about each school from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp).





### EdGap data

All socioeconomic data (household income, unemployment, adult educational attainment, and family structure) are from the Census Bureau's American Community Survey. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) report that ACT and SAT score data is from each state's department of education or some other public data release. The nature of the other public data release is not known.

The quality of the census data and the department of education data can be assumed to be reasonably high. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) do not indicate that they processed the data in any way. The data were assembled by the [EdGap.org](https://www.edgap.org/#5/37.875/-96.987) team, so there is always the possibility for human error. Given the public nature of the data, we would be able to consult the original data sources to check the quality of the data if we had any questions.

### School information data

The school information data is from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp). This data set consists of basic identifying information about schools and can be assumed to be of reasonably high quality. As for the EdGap.org data, the school information data is public, so we would be able to consult the original data sources to check the quality of the data if we had any questions.


## Load the data

Load the EdGap
 data set

In [96]:
edgap = pd.read_excel('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/education/EdGap_data.xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Load the school information data

In [97]:
!wget https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0

--2023-04-24 23:48:55--  https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv [following]
--2023-04-24 23:48:55--  https://www.dropbox.com/s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc60a7a688831cbbd9a077dae82a.dl.dropboxusercontent.com/cd/0/inline/B62j9lL7h9GZzKFJ5ZmVOybniOHn39pRHeIDJDkWnxH3R8zp3Q5rbQQbRjuTj7Cc6nnKanBR3O7Dt_xUa2Xi3eeO5qiA5aJvLur7Z0i-QQO9XVhHRwGDXH7fHK8SuflVBnNkQtSEveXOWQwqbYBK2HeN0vN__foWzBvNIXwQx8Q7uQ/file# [following]
--2023-04-24 23:48:56--  https://uc60a7a688831cbbd9a077dae82a.dl.dropboxusercontent.com/cd/0/inline/B62j9lL7h

In [98]:
school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding= 'unicode_escape')

<ipython-input-98-54e0747c4ccf>:1: DtypeWarning: Columns (6,9,14,15,18,19,21,22,25,26,29,31,35,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding= 'unicode_escape')


## Inspect the contents of each data set.

In this section we are inspects the contents of each data set, starting off by looking at the head of each data frame.

In [99]:
edgap.head()

,NCESSCH School ID,CT Unemployment Rate,CT Pct Adults with College Degree,CT Pct Childre In Married Couple Family,CT Median Household Income,School ACT average (or equivalent if SAT score),School Pct Free and Reduced Lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [100]:
school_info.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,...,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2016-2017,1,ALABAMA,AL,Sequoyah Sch - Chalkville Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
1,2016-2017,1,ALABAMA,AL,Camps,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
2,2016-2017,1,ALABAMA,AL,Det Ctr,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
3,2016-2017,1,ALABAMA,AL,Wallace Sch - Mt Meigs Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
4,2016-2017,1,ALABAMA,AL,McNeel Sch - Vacca Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported


We are using the `info` method to check the data types, size of the data frame, and numbers of missing values.

In [101]:
edgap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7986 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   NCESSCH School ID                                7986 non-null   int64  
 1   CT Unemployment Rate                             7972 non-null   float64
 2   CT Pct Adults with College Degree                7973 non-null   float64
 3   CT Pct Childre In Married Couple Family          7961 non-null   float64
 4   CT Median Household Income                       7966 non-null   float64
 5   School ACT average (or equivalent if SAT score)  7986 non-null   float64
 6   School Pct Free and Reduced Lunch                7986 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 436.9 KB


It appears that there is some missing points in the school_info()

In [102]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102183 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102183 non-null  object 
 1   FIPST                102183 non-null  int64  
 2   STATENAME            102183 non-null  object 
 3   ST                   102183 non-null  object 
 4   SCH_NAME             102183 non-null  object 
 5   LEA_NAME             102183 non-null  object 
 6   STATE_AGENCY_NO      102183 non-null  object 
 7   UNION                2533 non-null    float64
 8   ST_LEAID             102183 non-null  object 
 9   LEAID                102183 non-null  object 
 10  ST_SCHID             102183 non-null  object 
 11  NCESSCH              102181 non-null  float64
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3         

It appears that there is some missing points in the school_info()

## Convert data types, if necessary


We need to join two DataFrames using the school identity as the key, which is represented by the NCESSCH column. However, the column has different names in the two DataFrames and is stored as an int64 in one DataFrame and a float64 in the other.

To merge the two DataFrames, we'll first cast the NCESSCH column in the DataFrame with the float64 datatype to an int64. Before casting, we must drop any rows where the NCESSCH value is NaN.


In [103]:
school_info = school_info[school_info['NCESSCH'].isna() == False]

In [104]:
school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')

<ipython-input-104-006ea9997674>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')


In [105]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102181 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102181 non-null  object 
 1   FIPST                102181 non-null  int64  
 2   STATENAME            102181 non-null  object 
 3   ST                   102181 non-null  object 
 4   SCH_NAME             102181 non-null  object 
 5   LEA_NAME             102181 non-null  object 
 6   STATE_AGENCY_NO      102181 non-null  object 
 7   UNION                2531 non-null    float64
 8   ST_LEAID             102181 non-null  object 
 9   LEAID                102181 non-null  object 
 10  ST_SCHID             102181 non-null  object 
 11  NCESSCH              102181 non-null  int64  
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3         

## Select relevant subsets of the data

Since the school information data set contains a lot of information, we only need the year school identity, location, and school type information.

In [106]:
school_info = school_info[['SCHOOL_YEAR', 'NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL']]

In [107]:
school_info.head()

,SCHOOL_YEAR,NCESSCH,MSTATE,MZIP,SCH_TYPE_TEXT,LEVEL
0,2016-2017,10000200277,AL,35220,Alternative School,High
1,2016-2017,10000201667,AL,36057,Alternative School,High
2,2016-2017,10000201670,AL,36057,Alternative School,High
3,2016-2017,10000201705,AL,36057,Alternative School,High
4,2016-2017,10000201706,AL,35206,Alternative School,High


## Rename columns

Renaming the columns to follow best practices of being lowercase, snake_case

In [108]:
edgap = edgap.rename(columns={"NCESSCH School ID":"id", 
              "CT Pct Adults with College Degree":"percent_college",        
              "CT Unemployment Rate":"rate_unemployment", 
              "CT Pct Childre In Married Couple Family":"percent_married",
              "CT Median Household Income":"median_income",
              "School ACT average (or equivalent if SAT score)":"average_act",
              "School Pct Free and Reduced Lunch":"percent_lunch"})

In [109]:
school_info = school_info.rename(columns={'SCHOOL_YEAR':'year', 
                                          'NCESSCH':'id', 
                                          'MSTATE':'state',
                                          'MZIP':'zip_code',
                                          'SCH_TYPE_TEXT':'school_type',
                                          'LEVEL':'school_level'})

In [110]:
edgap.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [111]:
school_info.head()

,year,id,state,zip_code,school_type,school_level
0,2016-2017,10000200277,AL,35220,Alternative School,High
1,2016-2017,10000201667,AL,36057,Alternative School,High
2,2016-2017,10000201670,AL,36057,Alternative School,High
3,2016-2017,10000201705,AL,36057,Alternative School,High
4,2016-2017,10000201706,AL,35206,Alternative School,High


## Join data frames 

Here we are joing the two data sets with a left join 

In [112]:
edgap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7986 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7986 non-null   int64  
 1   rate_unemployment  7972 non-null   float64
 2   percent_college    7973 non-null   float64
 3   percent_married    7961 non-null   float64
 4   median_income      7966 non-null   float64
 5   average_act        7986 non-null   float64
 6   percent_lunch      7986 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 436.9 KB


In [113]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102181 entries, 0 to 102182
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   year          102181 non-null  object
 1   id            102181 non-null  int64 
 2   state         102181 non-null  object
 3   zip_code      102181 non-null  object
 4   school_type   102179 non-null  object
 5   school_level  102179 non-null  object
dtypes: int64(1), object(5)
memory usage: 5.5+ MB


In [114]:
df = edgap.merge(school_info, how = 'left', on='id')

In [115]:
df.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,2016-2017,DE,19804,Regular School,High
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,2016-2017,DE,19709,Regular School,High
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,2016-2017,DE,19709,Regular School,High
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,2016-2017,DE,19958,Regular School,High
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,2016-2017,DE,19934,Regular School,High


In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7986 entries, 0 to 7985
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7986 non-null   int64  
 1   rate_unemployment  7972 non-null   float64
 2   percent_college    7973 non-null   float64
 3   percent_married    7961 non-null   float64
 4   median_income      7966 non-null   float64
 5   average_act        7986 non-null   float64
 6   percent_lunch      7986 non-null   float64
 7   year               7898 non-null   object 
 8   state              7898 non-null   object 
 9   zip_code           7898 non-null   object 
 10  school_type        7898 non-null   object 
 11  school_level       7898 non-null   object 
dtypes: float64(6), int64(1), object(5)
memory usage: 811.1+ KB


##Identifying and Eliminating NaN/Out of bounds Values

Here we are filtering, removing and eliminating specifc values.

Filtering to only have "school_level == High"

In [117]:
df = df.loc[df['school_level'] == 'High']

Filtering the average act to be greater than or equal to 1 since anything less is not needed. 

In [118]:
df = df.loc[df['average_act' ] >= 1]

Similarly we are filtering percent lunch to only include values greater than zero 

In [119]:
df.loc[df['percent_lunch'] < 0, 'percent_lunch'] = np.nan 

In [120]:
df.isna().sum()

id                    0
rate_unemployment    12
percent_college      11
percent_married      20
median_income        16
average_act           0
percent_lunch        20
year                  0
state                 0
zip_code              0
school_type           0
school_level          0
dtype: int64

In [121]:
df.isna().mean().round(4)*100

id                   0.00
rate_unemployment    0.17
percent_college      0.15
percent_married      0.28
median_income        0.22
average_act          0.00
percent_lunch        0.28
year                 0.00
state                0.00
zip_code             0.00
school_type          0.00
school_level         0.00
dtype: float64

##Train Test split

##### $\rightarrow$ In the matrix predictor variable `x` is all columns except `id` and `average_act` 

#### Get input and output variables.

In [122]:
x = df[df.columns.difference(['id','average_act'])]

In [123]:
x.head()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,school_level,school_type,state,year,zip_code
0,42820.0,0.445283,0.066901,0.346495,0.117962,High,Regular School,DE,2016-2017,19804
1,89320.0,0.662765,0.112412,0.767619,0.063984,High,Regular School,DE,2016-2017,19709
2,84140.0,0.701864,0.096816,0.713090,0.056460,High,Regular School,DE,2016-2017,19709
3,56500.0,0.692062,0.296960,0.641283,0.044739,High,Regular School,DE,2016-2017,19958
4,54015.0,0.640060,0.262641,0.834402,0.077014,High,Regular School,DE,2016-2017,19934


##### $\rightarrow$ Defining the output variable `y` to be `average_act`.

In [124]:
y = df[['average_act']]

In [125]:
y.head()

,average_act
0,20.433455
1,19.498168
2,19.554335
3,17.737485
4,18.245421


#### Train and test splits

##### $\rightarrow$ Here we are splitting the data into training and testing sets. Keeping 20% of the data for the test set & rest for the train

In [126]:
x_train, x_test, y_train, y_test = train_test_split(x, y,  test_size=.2, random_state = 1)

In [127]:
print(x_train.shape, x_test.shape)

(5781, 10) (1446, 10)


#### Imputation

Defining an imputer to use later on called "imputer"

In [128]:
imputer = IterativeImputer()

In [129]:
imputer.fit(x_train.loc[:,'median_income':'rate_unemployment'])

IterativeImputer()

Imputing the missing values in the training data.

In [130]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5781 entries, 3663 to 5736
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   median_income      5766 non-null   float64
 1   percent_college    5770 non-null   float64
 2   percent_lunch      5764 non-null   float64
 3   percent_married    5763 non-null   float64
 4   rate_unemployment  5769 non-null   float64
 5   school_level       5781 non-null   object 
 6   school_type        5781 non-null   object 
 7   state              5781 non-null   object 
 8   year               5781 non-null   object 
 9   zip_code           5781 non-null   object 
dtypes: float64(5), object(5)
memory usage: 496.8+ KB


The line of code below is filling missing values in the 'median_income' to 'rate_unemployment' columns of the pandas dataframe x_train using an imputer object. 

In [131]:
x_train.loc[:, 'median_income':'rate_unemployment'] = imputer.transform(x_train.loc[:,'median_income':'rate_unemployment'])

Now missing values in train should be zero

In [132]:
x_train.isna().sum()

median_income        0
percent_college      0
percent_lunch        0
percent_married      0
rate_unemployment    0
school_level         0
school_type          0
state                0
year                 0
zip_code             0
dtype: int64

Now doing something similar for the Test data set. First checking missing values, filling and rechecking

In [133]:
x_test.isna().sum()

median_income        1
percent_college      0
percent_lunch        3
percent_married      2
rate_unemployment    0
school_level         0
school_type          0
state                0
year                 0
zip_code             0
dtype: int64

In [134]:
x_test.loc[:,'median_income':'rate_unemployment'] = imputer.transform(x_test.loc[:,'median_income':'rate_unemployment'])

In [135]:
x_test.isna().sum()

median_income        0
percent_college      0
percent_lunch        0
percent_married      0
rate_unemployment    0
school_level         0
school_type          0
state                0
year                 0
zip_code             0
dtype: int64

##Joining (X & Y) Train & Test

This line of code is joining two pandas dataframes x_train and y_train horizontally based on their index and creating a new dataframe df_train.

In [136]:
df_train = x_train.join(y_train)

In [137]:
df_train.head()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,school_level,school_type,state,year,zip_code,average_act
3663,41793.0,0.602419,0.542056,0.574034,0.111111,High,Regular School,NJ,2016-2017,7306,16.538462
1689,38173.0,0.469225,0.339655,0.711429,0.135246,High,Regular School,IN,2016-2017,47567,20.367521
5852,39635.0,0.567361,0.270175,0.694514,0.083419,High,Regular School,PA,2016-2017,15853,20.347985
3288,40978.0,0.467614,0.315556,0.766901,0.062531,High,Regular School,MO,2016-2017,64644,21.600000
378,36875.0,0.604470,0.548410,0.803435,0.071429,High,Regular School,FL,2016-2017,34669,21.056166


Similar to what we did for df_train we are doing it to df_test

In [138]:
df_test = x_test.join(y_test)

In [139]:
df_test.head()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,school_level,school_type,state,year,zip_code,average_act
2804,52833.0,0.564717,0.226481,0.823245,0.100518,High,Regular School,MI,2016-2017,49112,21.000000
4162,62411.0,0.537197,0.677895,0.313253,0.095582,High,Regular School,NY,2016-2017,11413,16.245421
5411,63938.0,0.781818,0.561431,0.523810,0.096433,High,Regular School,PA,2016-2017,15222,18.345543
4171,25625.0,0.361014,0.625239,0.317358,0.168471,High,Regular School,NY,2016-2017,11103,18.663004
1950,46350.0,0.602669,0.358377,0.641444,0.089737,High,Regular School,KY,2016-2017,42025,20.000000
